# An introduction to I-24 MOTION trajectory data

## Useful links
- I24 MOTION website: https://i24motion.org/
- Request for data access: https://i24motion.org/data
- Data documentation: https://github.com/I24-MOTION/I24M_documentation
- VT tools: https://github.com/I24-MOTION/VT_tools 
- Improvement tracker: https://github.com/I24-MOTION/I24M_improvement_tracker

## Directory structure
```
├── data_demo
│   ├── INCEPTION.22-11-22.tutorial.json
├── I24_tutorial_code
│   ├── intro_trajectory_data.ipynb
```

## This tutorial will cover:
- Load JSON file using an iterative JSON parser
- Trajectory data schema
- Compute derivative quantities (e.g., speed)
- Visualize a trajectory
- Plot a time-space diagram

## Import packages

In [ ]:
import ijson
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.ticker as mticker
from mpl_toolkits.axes_grid1 import make_axes_locatable
import datetime

### Read a JSON file using an iterative JSON parser

In [ ]:
# This data file contains approximately 41,000 trajectories
input_filename = "INCEPTION.22-11-22.tutorial.json"
file_path = os.path.join("../data_demo/", input_filename)

i = 0
# Select one trajectory from westbound that is more than 10 sec long
with open(file_path, 'r') as input_file:
    parser = ijson.items(input_file, 'item', use_float=True)
    for record in parser:
        if record["direction"] == -1 and (record["last_timestamp"] - record["first_timestamp"] > 10):
            print("Found a trajectory example")
            break

In [ ]:
print("TRAJECTORY KEYS:", list(record.keys()))
print("\n\n")
for key, val in record.items():
    if isinstance(val, list):
        print(f"{key}:   {val[:min(len(val), 20)]}")
    else:
        print(f"{key}:   {val}")

### Compute the derivative quantities

In [ ]:
# Compute the longitudinal speed using numerical differentiation of the x_position, 
#   i.e., v = dx/dt. np.diff() computes the first-order numerical derivative.
# Multiply by record["direction"] to indicate the direction, 
#   i.e., negative speed means traveling westbound, and positive speed means traveling eastbound.
speed = np.diff(record["x_position"])/np.diff(record["timestamp"])*record["direction"]

# Speed array is truncated by one after the numerical differentiation np.diff(). 
# Append the first item to the array such that the speed array is the same length as the x_position array.
speed = np.append(speed[0], speed) 

# Convert unit from ft/sec to mph
speed *= 0.681818

# Print the first 10 records of speed
print(f"Speed (mph): {speed[:10]}")

In [ ]:
# Similarly, compute the 2nd-order numerical differentiation of the x_position to obtain acceleration. 
# np.diff(record["x_position"], n=2) means the second order differentation of x_position.
# The unit for acceleration is ft/sec^2.
accel = np.diff(record["x_position"], n=2)/(np.diff(record["timestamp"][:-1])**2)*record["direction"]
print(f"Acceleration (ft/sec^2): {accel[:10]}")

### Visualize a trajectory

In [ ]:
import time

In [ ]:
# Set up the plot
plt.rc('font', family='serif', size=14)
fig, ax = plt.subplots(figsize=(10,3))
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="3%", pad=0.05)

# Define the color range
jet = plt.cm.jet
colors = [jet(x) for x in np.linspace(1, 0.5, 256)]
green_to_red = LinearSegmentedColormap.from_list('GreenToRed', colors, N=256)

# -------------------------------------------------------------------
# Plot the position (y-axis) vs. time (x-axis), colored by the speed.
# Position in ft is converted to mile-marker by dividing 5280.
trajectory_times = record["timestamp"]
trajectory_xvals = np.array(record["x_position"])/5280
im = ax.scatter(trajectory_times, trajectory_xvals, c=speed, cmap=green_to_red, vmin=0, vmax=80, marker ='s', s=5)
plt.colorbar(im, cax=cax).set_label('Speed (mph)', rotation=90, labelpad=20)
ax.set_xlabel("Time")
ax.set_ylabel("Mile marker")
# -------------------------------------------------------------------

# Update x-axis time to readable format
ticks_loc = ax.get_xticks().tolist()
ax.xaxis.set_major_locator(mticker.FixedLocator(ticks_loc))
x_datetime = [datetime.datetime.fromtimestamp(ts) for ts in ticks_loc]
labels = [d.strftime('%H:%M:%S') for d in x_datetime]
ax.set_xticklabels(labels, rotation=45)

# Invert vertically
ax.invert_yaxis()


### Plot a time-space diagram (approx. 8:20-8:23 AM, Westbound)

In [ ]:
# Set up the plot
plt.rc('font', family='serif', size=14)
fig, ax = plt.subplots(figsize=(10,3))
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="3%", pad=0.05)

# Define the color range
jet = plt.cm.jet
colors = [jet(x) for x in np.linspace(1, 0.5, 256)]
green_to_red = LinearSegmentedColormap.from_list('GreenToRed', colors, N=256)

# -------------------------------------------------------------------
# Iterate over each trajectory record across the file, compute the speed, and plot the position and speed simultaneously
# ijson allows us to read and plot each trajectory one by one instead of loading them all into the memory
t = time.time()
i = 0
with open(file_path, 'r') as input_file:
    parser = ijson.items(input_file, 'item', use_float=True)
    for record in parser:  
        # select westbound trajectories to plot
        if record["direction"] == -1: 
            position = np.array(record["x_position"]) / 5280
            speed = np.diff(record["x_position"]) / np.diff(record["timestamp"])
            speed = np.append(speed[0], speed)
            speed = -speed * 0.681818 # convert unit, and
            im = ax.scatter(record["timestamp"], position, c=speed, cmap=green_to_red, vmin=0, vmax=80, s=0.1)
            i += 1
            if i % 1000 == 0:
                print(f"Plotted {i} trajectories")
            if i > 5000:
                break
print(f"Elapsed: {time.time() - t}")
# -------------------------------------------------------------------

# Update colorbar and axes labels
plt.colorbar(im, cax=cax).set_label('Speed (mph)', rotation=90, labelpad=20)
ax.set_xlabel("Time")
ax.set_ylabel("Mile marker")

# Update x-axis time to readable format
ticks_loc = ax.get_xticks().tolist()
ax.xaxis.set_major_locator(mticker.FixedLocator(ticks_loc))
x_datetime = [datetime.datetime.fromtimestamp(ts) for ts in ticks_loc]
labels = [d.strftime('%H:%M:%S') for d in x_datetime]
ax.set_xticklabels(labels, rotation=45)

# Invert vertically
ax.invert_yaxis()
plt.show()

#### In order to plot these additional data, you will need to download the appropriate file from i24motion.org/data using your account credentials. Download it to the same directory where you found `INCEPTION.22-11-22.tutorial.json`, or else change the path location.